# Business Understanding
De organisatie wilt graag inzicht verkrijgen in hun werknemersverloop (attrition). Er zijn meerdere databronnen beschikbaar gesteld in de vorm van CSV-bestanden. De variabelen uit deze bestanden zullen worden geanalyseerd, waaruit een aantal relevante variabelen geselecteerd worden voor een voorspellend model. Voor de scope en richting van het onderzoek zijn onderstaand een tweetal doelstellingen beschreven.

* Inzicht krijgen in welke variabelen een relatie met elkaar hebben
* Een voorspellend model bouwen om werknemersverloop te minderen

Tegelijkertijd zal het onderzoek een succes zijn als:

* De organisatie preventief kan handelen om werknemersverloop te minderen

In [64]:
# Importeren van de benodigde libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import datetime
from sklearn.preprocessing import StandardScaler


# Data Understanding
De aangeleverde data komen uit twee verschillende bronnen; vanuit HR (bestaand) en enquêtes (additioneel). Tevens is een data dictionary meegeleverd, zodat uniforme definities gehanteerd worden. Onderstaand een beknopte opsomming van de data.

* Standaard HR-data waarin belangrijke werknemers data (anoniem) staat
* Tweemaal enquête data; één van de werknemers en één van de managers
* Kloktijden met de start en eind timestamps van de shifts

### Definities van de variabelen

| Variabel                 | Omschrijving                                       | Niveau |
| ------------------------ | -------------------------------------------------- | ------ |
| Age                      | Leeftijd werknemer                                 |        |
| Attrition                | De werknemer is vorig jaar weggegaan               |        |
| BusinessTravel           | Frequentie van zakelijke reizen afgelopen jaar     |        |
| Department               | Afdeling binnen het bedrijf                        |        |
| DistanceFromHome         | Afstand vanaf huis in kilometers (km's)            |        |
| Education                | Educatie niveau                                    | 1. Below college <br> 2. College <br> 3. Bachelor <br> 4. Master <br> 5. Doctor |
| EducationField           | Educatie vakgebied                                 |        |
| EmployeeCount            | Aantal werknemers                                  |        |
| EmployeeNumber           | Unieke werknemersnummer (ID)                       |        |
| EnvironmentSatisfaction  | Tevredenheidsniveau van de werkomgeving            | 1. Low <br> 2. Medium <br> 3. High <br> 4. Very High |
| Gender                   | Geslacht                                           |        |
| JobInvolvement           | Niveau van aansluiting tussen werknemer en functie | 1. Low <br> 2. Medium <br> 3. High <br> 4. Very High |
| JobLevel                 | Niveau van functie binnen de organisatie           |        |
| JobRole                  | Naam functie                                       |        |
| JobSatisfaction          | Tevredenheidsniveau over werk                      | 1. Low <br> 2. Medium <br> 3. High <br> 4. Very High |
| MaritalStatus            | Burgelijke staat                                   |        |
| MonthlyIncome            | Maandelijk inkomen in roepies (valuta India)       |        |
| NumCompaniesWorked       | Totaal aantal werkgevers van werknemer             |        |
| Over18                   | Of de werknemer boven de 18 is of niet             |        |
| PercentSalaryHike        | Procentuele salarisverhoging afgelopen jaar        |        |
| PerformanceRating        | Resultaatbeoordeling afgelopen jaar                | 1. Low <br> 2. Good <br> 3. Excellent <br> 4. Outstanding |
| RelationshipSatisfaction | Tevredenheidsniveau van relatie                    | 1. Low <br> 2. Medium <br> 3. High <br> 4. Very High |
| StandardHours            | Standaard werkuren van werknemer                   |        |
| StockOptionLevel         | Aandelenniveau van werknemer                       |        |
| TotalWorkingYears        | Totaal aantal werkjaren van werknemer              |        |
| TrainingTimesLastYear    | Aantal training voor werknemer afgelopen jaar      |        |
| WorkLifeBalance          | Niveau van werk/privé balans                       | 1. Bad <br> 2. Good <br> 3. Better <br> 4. Best |
| YearsAtCompany           | Totaal aantal werkjaren bij de organisatie         |        |
| YearsSinceLastPromotion  | Aantal jaren sinds laatste promotie                |        |
| YearsWithCurrentManager  | Aantal jaar onder huidige manager                  |        |

In [45]:
# Data inlezen
employee_survey_data = pd.read_csv('employee_survey_data.csv')
general_data = pd.read_csv('general_data.csv')
manager_survey_data = pd.read_csv('manager_survey_data.csv')
in_time = pd.read_csv('in_time.csv')
out_time = pd.read_csv('out_time.csv')

# Kolomnamen aanpassen
in_time.rename(columns={in_time.columns[0]: 'EmployeeID'}, inplace=True)
out_time.rename(columns={out_time.columns[0]: 'EmployeeID'}, inplace=True)

# Data samenvoegen
data = pd.merge(general_data, employee_survey_data, on='EmployeeID')
data = pd.merge(data, manager_survey_data, on='EmployeeID')

In [63]:
# Kloktijden omzetten naar datetime
if 'Time' in in_time.columns:
    in_time['Time'] = pd.to_datetime(in_time['Time'], format='%d-%m-%Y %H:%M:%S')
if 'Time' in out_time.columns:
    out_time['Time'] = pd.to_datetime(out_time['Time'], format='%d-%m-%Y %H:%M:%S')

print(in_time.head())

   EmployeeID  2015-01-01 2015-01-02 2015-01-05 2015-01-06 2015-01-07  \
0           1         NaN   09:43:45   10:08:48   09:54:26   09:34:31   
1           2         NaN   10:15:44   10:21:05        NaN   09:45:17   
2           3         NaN   10:17:41   09:50:50   10:14:13   09:47:27   
3           4         NaN   10:05:06   09:56:32   10:11:07   09:37:30   
4           5         NaN   10:28:17   09:49:58   09:45:28   09:49:37   

  2015-01-08 2015-01-09 2015-01-12 2015-01-13  ...  2015-12-18 2015-12-21  \
0   09:51:09   10:09:25   09:42:53   10:13:06  ...         NaN   09:55:29   
1   10:09:04   09:43:26   10:00:07   10:43:29  ...    10:37:17   09:49:02   
2   10:03:40   10:05:49   10:03:47   10:21:26  ...    10:15:14   10:10:28   
3   10:02:08   10:08:12   10:13:42   09:53:22  ...    10:17:38   09:58:21   
4   10:19:44   10:00:50   10:29:27   09:59:32  ...    09:58:35   10:03:41   

  2015-12-22 2015-12-23 2015-12-24 2015-12-25 2015-12-28 2015-12-29  \
0   10:04:06   10:14:27   1

### Belangrijke overwegingen

1. De vijf CSV-bestanden bevatten de unieke werknemers-ID in de eerste kolom voor het samenvoegen van de data
2. 